In [101]:
import math
import numpy as np
def _tiny():
    return np.finfo('float64').eps

def calculate_d3r(rt, last_vt, last_ddt):
    x = (rt - 0.5*last_vt) / (last_ddt + _tiny())
    y = ((last_ddt**2)*(rt - 0.5*last_vt) - 0.5*last_vt*(rt**2)) / (last_ddt**3 + _tiny())
    return (x,y)
def calculate_dsr(rt, last_vt, last_wt):
    delta_vt = rt - last_vt
    delta_wt = rt**2 - last_wt
    return (last_wt * delta_vt - 0.5 * last_vt * delta_wt) / ((last_wt - last_vt**2)**(3/2) + _tiny())

def calculate_d3r(rt, last_vt, last_ddt):
    x = (rt - 0.5*last_vt) / (last_ddt + _tiny())
    y = ((last_ddt**2)*(rt - 0.5*last_vt) - 0.5*last_vt*(rt**2)) / (last_ddt**3 + _tiny())
    return (x,y)
class RiskAdjusteMetric:
    def __init__(self, last_vt,last_wt, last_ddt, last_dt2,decay_rate):
      self.last_vt = last_vt
      self.last_wt = last_wt
      self.decay_rate=decay_rate
      self.last_ddt = last_ddt
      self.last_dt2 = last_dt2
      self.last_sr = 0
      self.last_ddr = 0
      
      
    def _tiny():
        return np.finfo('float64').eps



    def update_metrics(self, rt):
        rt = np.log(rt+1) 

        dsr = calculate_dsr(rt, self.last_vt, self.last_wt)
        d3r_cond1, d3r_cond2 = calculate_d3r(rt, self.last_vt, self.last_ddt)
        d3r = d3r_cond1 if rt > 0 else d3r_cond2

        self.last_vt += self.decay_rate * (rt - self.last_vt)
        self.last_wt += self.decay_rate * (rt**2 - self.last_wt)

        self.last_dt2 += self.decay_rate * (np.minimum(rt, 0)**2 - self.last_dt2)
        self.last_ddt = math.sqrt(self.last_dt2)
        self.last_sr += self.decay_rate * dsr
        self.last_ddr += self.decay_rate * d3r
        return dsr*self.dec, d3r


In [79]:
from pandas import read_csv


df_dow =read_csv('./data/dow.csv')
df =df_dow.copy()
df = df_dow[df_dow.tic.isin( ['AAPL'])]

In [88]:
returns = df['close'].pct_change()

In [89]:
returns[1:]

25        0.006711
50        0.003336
75        0.003322
100       0.006622
125      -0.009868
            ...   
244447    0.003974
244477   -0.004398
244507    0.005080
244537    0.001187
244567    0.022872
Name: close, Length: 8752, dtype: float64

In [109]:
estimator = RiskAdjusteMetric(0,0,0,0,0.0001)
for i,item in enumerate(returns[1:]):
 print(estimator.update_metrics(item)[0])


0.0
0.003736926172025276
0.0031162415709022106
0.0027561419886066372
-0.017898140471091964
-0.06292955465933703
-0.006077814988806377
6.088078501157176e-07
-0.0010728044710329376
0.0033912148801499265
-0.00462143347474674
-0.0025275198733680456
0.010116864292227017
-0.0025490675542501496
0.0015039420489130863
0.0007071936253818152
0.00034547835174533054
-0.0032200159060444267
0.0014072860858559176
0.0020926143081200555
3.8415419896751517e-07
-0.0009007421888569738
0.0016486502965467526
0.0019088387108114092
-0.0005761625523598021
-0.0032306434328702566
-0.0005637821332038169
0.0032227692582684213
-0.0005304645537713609
0.0011127317395179714
-0.0005235519290738667
3.0575860770202904e-07
-0.0008174815696756133
-0.0005305624431033753
0.0011202497412165454
-0.002009165527058345
0.0005453439583926187
0.0016700263670814202
-0.0009994871035642209
0.0010648616219577503
0.0005134709006279476
-0.000984441676176038
0.0015807323864453467
0.0015102605175924076
0.00023855352670961987
0.0026623281554

In [105]:
df['close'].pct_change()[1:].mean()/df['close'].pct_change()[1:].std()

0.042591706397122536